$\textbf{Introduction}$

This is a study of automobile demand, as a function of disposable income, US mortage rates and the rate of change of automobile prices.

First lets read in the data from the spread sheets.

We will do a linear autoregression with variable lags to select the best lag structure.  Then we will compare the linear autogressive model with the neural net specification.

In [ ]:
#!pip install --upgrade pandas

In [1]:

import pandas as pd
from datetime import timedelta

df1 = pd.read_excel('AutoPrices.xls', header=0, names=['observation_date','CUSR0000SETA01'],
                   parse_dates=['observation_date']).iloc[10:]
df1.set_index('observation_date', inplace=True)
df1.index= pd.to_datetime(df1.index)

df2 = pd.read_excel('DAUPSA.xls', header=0, names=['observation_date','DAUPSA'],
                   parse_dates=['observation_date']).iloc[10:]
df2.set_index('observation_date', inplace=True)
df2.index= pd.to_datetime(df2.index)

df3 = pd.read_excel('DisposIncome.xls', header=0, names=['observation_date','DSPIC96'],
                    parse_dates=['observation_date']).iloc[10:]
df3.set_index('observation_date', inplace=True)
df3.index= pd.to_datetime(df3.index)



df4 = pd.read_excel('MORTGAGE15US.xls', header=0, names=['observation_date','MORTGAGE15US'], 
                    parse_dates=['observation_date']).iloc[10:]
df4.set_index('observation_date', inplace=True)
df4.index= pd.to_datetime(df4.index)
df4 = df4.resample('M').last()
df4.index=df4.index+timedelta(days=1)


In [2]:
df1_2 = pd.merge(df1, df2, left_index=True, right_index=True)
df1_2_3 =pd.merge(df1_2, df3, left_index=True, right_index=True)
df =pd.merge(df1_2_3, df4, left_index=True, right_index=True)
df

,CUSR0000SETA01,DAUPSA,DSPIC96,MORTGAGE15US
observation_date,,,,
1993-01-01,130.9,512.9,7237.6,7.68
1993-02-01,131.1,503.8,7271.8,7.37
1993-03-01,131.3,498.1,7249.2,7.02
1993-04-01,131.7,510.2,7286.8,7.01
1993-05-01,132.2,512.8,7276.3,6.91
...,...,...,...,...
2021-05-01,151.693,128.1,15598.1,2.31
2021-06-01,154.68,120.3,15517.7,2.27
2021-07-01,157.34,142.2,15617.6,2.34


In [3]:
df = df.rename({'CUSR0000SETA01':'AutoPrice', 'DAUPSA':'AutoProd','DSPIC96':'DisIncome', 'MORTGAGE15US':'MortRate'}, axis=1)
df

,AutoPrice,AutoProd,DisIncome,MortRate
observation_date,,,,
1993-01-01,130.9,512.9,7237.6,7.68
1993-02-01,131.1,503.8,7271.8,7.37
1993-03-01,131.3,498.1,7249.2,7.02
1993-04-01,131.7,510.2,7286.8,7.01
1993-05-01,132.2,512.8,7276.3,6.91
...,...,...,...,...
2021-05-01,151.693,128.1,15598.1,2.31
2021-06-01,154.68,120.3,15517.7,2.27
2021-07-01,157.34,142.2,15617.6,2.34


In [4]:
df = df.astype(dtype={'AutoPrice':float,'AutoProd':float,'DisIncome':float, 'MortRate':float})

In [5]:
df.head(20)

,AutoPrice,AutoProd,DisIncome,MortRate
observation_date,,,,
1993-01-01,130.9,512.9,7237.6,7.68
1993-02-01,131.1,503.8,7271.8,7.37
1993-03-01,131.3,498.1,7249.2,7.02
1993-04-01,131.7,510.2,7286.8,7.01
1993-05-01,132.2,512.8,7276.3,6.91
1993-06-01,132.3,489.5,7262.9,6.99
1993-07-01,132.9,468.1,7281.2,6.84
1993-08-01,133.4,457.1,7291.8,6.75
1993-09-01,133.6,441.7,7271.9,6.49


- lets start from 94


In [6]:
df_cur = df.iloc[12:] # shift one year later
df_prev = df.iloc[:-12] 
df.shape, df_cur.shape, df_prev.shape

((345, 4), (333, 4), (333, 4))

Now let's pick out the arrays and look at the data, we see the deep drop in Mortgage Rates after 2010

In [7]:
import matplotlib.pyplot as plt
df.plot(y='MortRate')
plt.grid(True)

ModuleNotFoundError: No module named 'matplotlib.artist'

## Data Transformation

In [8]:
import numpy as np
df_transformed = pd.DataFrame(columns=df.columns)
for col in df.columns:
    df_transformed[col] =  np.log(df_cur[col].values) - np.log(df_prev[col].values)
    print(df_cur[col].values.shape)
df_transformed.index = df_cur.index

(333,)
(333,)
(333,)
(333,)


In [9]:
! pip install --upgrade matplotlib

  Using cached matplotlib-3.7.1-cp39-cp39-win_amd64.whl (7.6 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mcnel\\anaconda4\\Lib\\site-packages\\matplotlib\\ft2font.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [10]:
import matplotlib.pyplot as plt
fig,axes = plt.subplots(nrows=3,figsize=(10,10))
ax = axes[0]
df_transformed.plot(ax=ax,title='Disposable Income', y='DisIncome')
ax.grid(True)

ax = axes[1]
df_transformed.plot(ax=ax,y='AutoProd', title='Automobile Production')
ax.grid(True)

ax = axes[2]
df_transformed.plot(ax=ax,y='AutoPrice', title='Automobile Prices')
ax.grid(True)
fig.tight_layout(pad=3.0)

ImportError: cannot import name 'axes' from 'matplotlib' (C:\Users\mcnel\anaconda4\lib\site-packages\matplotlib\__init__.py)

In [11]:
 df_transformed.iloc[12:]

,AutoPrice,AutoProd,DisIncome,MortRate
observation_date,,,,
1995-01-01,0.032718,-0.001736,0.038708,0.284283
1995-02-01,0.030481,-0.012928,0.038805,0.303020
1995-03-01,0.028253,0.031404,0.038250,0.204141
1995-04-01,0.030327,-0.036327,0.030809,0.086555
1995-05-01,0.028069,-0.065408,0.029413,-0.006390
...,...,...,...,...
2021-05-01,0.032890,0.969188,-0.046038,-0.181600
2021-06-01,0.051273,-0.156641,-0.034182,-0.143394
2021-07-01,0.061663,-0.418044,-0.033639,-0.101507


In [12]:
y = df_transformed.iloc[12:]['AutoProd']
normalized_y=(y-y.min())/(y.max()-y.min()) #[0,1]
#y = normalized_y

x = df_transformed.iloc[:-12]
normalized_x=(x-x.min())/(x.max()-x.min())
#x = normalized_y

In [13]:
ry = len(y)
ry

321

Lets do a split of in and out of sample with ols.  Let 85 percent of the sample be for estimaition and the last for the final test of root mean squared error.  He calculate the Hanan-Quinn information criteria for the insample performance with zero lags.

In [32]:
import math
import statsmodels.api as sm
nn = math.ceil(.85*len(y))

xin = x.iloc[:nn]
yin = y.iloc[:nn]

xout = x.iloc[nn:]
yout = y.iloc[nn:]

print(yin.shape)


mod = sm.OLS(yin.values, xin.values)
res = mod.fit()

B  = res.params

ehat = res.resid
rr = ehat.shape

#yhat_in = np.dot(xin, B)
#ehat = yin.values - np.squeeze(yhat_in)
sse = np.dot(ehat.T, ehat)/nn;
hqif =  -2 * np.log(sse) + 2 *len(B)*np.log(np.log(len(xin)))
hqif

(273,)


20.545635439557554

In [ ]:
sse

On the basis of the Hannan-Quinn criterion, we select a lag length of 4.
We do the regression for the insample and do the out of sample by multiplying the Bols to the out of sample xout to get 
the prediction yhat_out.  We calculate the error and the Root mean squared error.  We get the value of .1644.

In [33]:
x11 = lagmat(xin,maxlag=4, trim="forward", original='in');
x11out = lagmat(xout,maxlag=4, trim="forward", original='in');
    #print('x11',x11.shape)
    
y11 = yin.iloc[4:]
x21 = x11[4:]
Y = y11; X = x21;
    
Yout = yout[4:]
    
Xout = x11out[4:]
mod = sm.OLS(Y,X)
res = mod.fit()

B  = res.params

Yhat_out = np.dot(Xout,B)

error_out_ols = Yhat_out - Yout
#error_out = youyhat_out



In [34]:

rmse_ols =  (np.dot(error_out_ols.T,error_out_ols)/len(yout))**.5;
rmse_ols

0.8121230387987857

Now let's do a rolling window regression in which we preduct for one period forward, not for the full last 15 percent of the sample.  After we we revise our models for forecasting period by period.  

In [35]:
ny = len(y)
WL = 100;
TERM = ny - (WL+1)
y = np.array(y)
x = np.array(x)
Yhat_out_rw = np.zeros((ny-WL,1))
Error_out_rw = np.zeros((ny-WL,1))

In [36]:



for i in range(1,TERM):
    WLL = WL+(i-1)
    WLLp = WLL +1
    yy = y[i:WLL]
    xx = x[i:WLL,:]
    xjunk = x[WLLp,:]
    mod = sm.OLS(yy, xx)
    res = mod.fit()
    B  = res.params
    Yhat_out_rw[i] = np.dot(xjunk,B)
    Error_out_rw[i] = y[WLLp] - Yhat_out_rw[i]

    
    



In [37]:
re = len(Error_out_rw)
re

221

Now lets see if the simple neural network does better out of sample. We feed in the same in and out of sample x and y variables as well as a simple neural net structure

hidden_layer_sizes: The number of neurons in each hidden layer, represented as a tuple of integers.
activation: The activation function used in the hidden layers, such as 'relu', 'tanh', or 'logistic'.
solver: The algorithm used to optimize the weights, such as 'lbfgs', 'sgd', or 'adam'.
alpha: The regularization strength, a positive float.
batch_size: The number of samples per weight update in SGD training, an integer.
learning_rate: The learning rate schedule for weight updates, either 'constant', 'invscaling', or 'adaptive'.
learning_rate_init: The initial learning rate used.
power_t: The exponent for the inverse scaling learning rate, a float.
max_iter: The maximum number of iterations for training, an integer.
shuffle: Whether to shuffle samples before each iteration, a boolean.
random_state: The seed for the random number generator, an integer.

In [38]:
#!pip install --upgrade scikit-learn
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

#X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, noise=0.1, random_state=42)

X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
Xout = np.concatenate((np.ones((Xout.shape[0], 1)), Xout), axis=1)
rx, cx = X.shape

#regr = MLPRegressor(random_state=0, max_iter=50000, hidden_layer_sizes=(4,), validation_fraction=0.1, 'invscaling',solver='adam').fit(X, Y)

regr = MLPRegressor(hidden_layer_sizes=(3*rx,2*rx,rx), max_iter=50000, random_state=0, learning_rate='constant',
                    solver='lbfgs', validation_fraction=0.3, activation='tanh', alpha=.25).fit(X, Y)
Yhat_out_nn = regr.predict(Xout)

print(Yhat_out_nn.shape)
error_nn = Yhat_out_nn - Yout
rr = error_nn.shape
sse_nn = np.dot(error_nn.T,error_nn)
rmsq_nn= (sse_nn/rr)**.5
rmsq_nn



(44,)


array([0.8483917])

In [39]:
regr = MLPRegressor(hidden_layer_sizes=(3*rx,2*rx,rx), max_iter=50000, random_state=0, learning_rate='constant',
                    solver='lbfgs', validation_fraction=0.3, activation='tanh', alpha=.25).fit(X, Y)
Yhat_out_nn = regr.predict(Xout)

print(Yhat_out_nn.shape)
error_nn = Yhat_out_nn - Yout
sse_nn = np.dot(error_nn.T,error_nn)/rr
rmsq_nn= sse_nn**.5
rmsq_nn


(44,)


array([0.8483917])

In [40]:
regr = MLPRegressor(hidden_layer_sizes=(3*rx,2*rx,rx), max_iter=50000, random_state=0, learning_rate='constant',
                    solver='lbfgs', validation_fraction=0.1, activation='tanh', alpha=.5).fit(X, Y)
Yhat_out_nn = regr.predict(Xout)

print(Yhat_out_nn.shape)
error_nn = Yhat_out_nn - Yout
sse_nn = np.dot(error_nn.T,error_nn)/rr
rmsq_nn= sse_nn**(.5)
rmsq_nn


(44,)


array([1.0028008])

In [41]:
regr = MLPRegressor(hidden_layer_sizes=(3*rx,2*rx,rx), max_iter=50000, random_state=0, learning_rate='constant',
                    solver='lbfgs', validation_fraction=0.2, activation='tanh', alpha=.15).fit(X, Y)
Yhat_out_nn = regr.predict(Xout)

print(Yhat_out_nn.shape)
error_nn = Yhat_out_nn - Yout
sse_nn = np.dot(error_nn.T,error_nn)/rr
rmsq_nn= sse_nn**(.5)
rmsq_nn

(44,)


array([0.82984092])

In [ ]:
import tensorflow as tf

def create_mlp_model(input_dim, l1_reg):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(3*rx, input_dim=input_dim, activation='tanh',
                                    kernel_regularizer=tf.keras.regularizers.l1(l1_reg)))
    model.add(tf.keras.layers.Dense(2*rx, activation='tanh',
                                    kernel_regularizer=tf.keras.regularizers.l1(l1_reg)))
    model.add(tf.keras.layers.Dense(rx, activation='tanh',
                                    kernel_regularizer=tf.keras.regularizers.l1(l1_reg)))
    model.add(tf.keras.layers.Dense(1))
    return model

input_dim = X.shape[1]
mlp = create_mlp_model(input_dim, l1_reg=0.2)
mlp.compile(optimizer='Adam', loss='mean_squared_error')

history = mlp.fit(X, Y, epochs=1000, batch_size=32, validation_data=(Xout, Yout))


In [ ]:
history

In [ ]:
Yhat_out= mlp.predict(Xout)

Yhat_out = Yhat_out.ravel()
Yout = np.array(Yout)


ERROR = Yout - Yhat_out


In [ ]:
rr = ERROR.shape
RMSQ =   (np.dot(ERROR.T,ERROR)/rr)**.5

In [ ]:
RMSQ = (np.dot(ERROR.T,ERROR)/rr)**.5
RMSQ

In [ ]:
rmse_ols

In [ ]:

error_out_ols
ERROR


In [ ]:
import pandas as pd

# Assuming matrix1 has a date index and matrix2 does not



# Convert matrix2 into a DataFrame with the same index as matrix1
ERROR1 = pd.DataFrame(ERROR, index=error_out_ols.index)

# Concatenate the two matrices
ERRORboth = pd.concat([error_out_ols, ERROR1], axis=1)
ERRORboth.columns =['ERRORols', 'ERRORnn']


In [ ]:

ERRORboth.plot(y=['ERRORols', 'ERRORnn'],kind='line')
plt.grid(True)

In [ ]:
ERRORboth.plot
plt.show()